In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [13]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [14]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-01-31 17:13:53 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [15]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [16]:
from collections import defaultdict

In [17]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-31 17:13:55 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 707Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_171355/train.jsonl: file-u8iPFyFXJAp6Eo3zDKB3NlPS


2023-01-31 17:13:58.017 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675181637,
  "events": [
    {
      "created_at": 1675181637,
      "level": "info",
      "message": "Created fine-tune: ft-LDIyknMulwVBINhl3kaxpTO2",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-LDIyknMulwVBINhl3kaxpTO2",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675181637,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_171355/train.jsonl",
      "id": "file-u8iPFyFXJAp6Eo3zDKB3NlPS",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-01-31 20:48:06 INFO     Recorded fold matbench_expt_gap-0 successfully.
Input contains NaN.


...

2023-02-01 03:42:57 INFO     Recorded fold matbench_expt_gap-3 successfully.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 815Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_034257/train.jsonl: file-zX4sUwoEfc7go1XjA2jcMfSO


2023-02-01 03:43:01.074 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675219381,
  "events": [
    {
      "created_at": 1675219381,
      "level": "info",
      "message": "Created fine-tune: ft-Q2Eq6pbQVX4AWeL1sJdGv7y6",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-Q2Eq6pbQVX4AWeL1sJdGv7y6",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420710,
      "created_at": 1675219380,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_034257/train.jsonl",
      "id": "file-zX4sUwoEfc7go1XjA2jcMfSO",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-02-01 05:12:54 INFO     Recorded fold matbench_expt_gap-4 successfully.


In [18]:
task.scores

ValueError: invalid __array_struct__

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
